In [6]:
include("../src/extractdata.jl")
include("../src/scrap.jl")

get_tags (generic function with 2 methods)

In [8]:
word = "龙塘镇_(海口市)"
get_wiki_zh2en(word)

"Longtang,_Hainan"

In [17]:
r = HTTP.get(addr)

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
date: Thu, 30 Jun 2022 13:03:52 GMT
server: mw1355.eqiad.wmnet
x-content-type-options: nosniff
content-language: zh
vary: Accept-Encoding,Cookie,Accept-Language,Authorization
last-modified: Wed, 29 Jun 2022 01:24:17 GMT
content-type: text/html; charset=UTF-8
age: 680
x-cache: cp5007 miss, cp5016 hit/29
x-cache-status: hit-front
server-timing: cache;desc="hit-front", host;desc="cp5016"
strict-transport-security: max-age=106384710; includeSubDomains; preload
report-to: { "group": "wm_nel", "max_age": 86400, "endpoints": [{ "url": "https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&schema_uri=/w3c/reportingapi/network_error/1.0.0" }] }
nel: { "report_to": "wm_nel", "max_age": 86400, "failure_fraction": 0.05, "success_fraction": 0.0}
set-cookie: WMF-Last-Access=30-Jun-2022;Path=/;HttpOnly;secure;Expires=Mon, 01 Aug 2022 12:00:00 GMT, WMF-Last-Access-Global=30-Jun-2022;Path=/;Domain=.wikipedia.org;HttpOnly;secure;

In [21]:
addr = "https://zh.wikipedia.org/wiki/$word"
@time [HTTP.get(addr) for _ in 1:10]
@time [get_wiki_zh2en(word) for _ in 1:10];

  7.407380 seconds (47.72 k allocations: 5.507 MiB, 0.52% compilation time)
  8.091447 seconds (281.96 k allocations: 24.690 MiB, 0.11% gc time, 0.61% compilation time)


10-element Vector{String}:
 "Longtang,_Hainan"
 "Longtang,_Hainan"
 "Longtang,_Hainan"
 "Longtang,_Hainan"
 "Longtang,_Hainan"
 "Longtang,_Hainan"
 "Longtang,_Hainan"
 "Longtang,_Hainan"
 "Longtang,_Hainan"
 "Longtang,_Hainan"

In [6]:
using HTTP
using Gumbo
using AbstractTrees
function get_wiki_zh2en(word)
    # 下载数据
    addr = "https://zh.wikipedia.org/wiki/$word"
    r = HTTP.get(addr)
    # 处理数据
    r_parsed = parsehtml(String(r.body))
    body = r_parsed.root[2]
    nav = get_tags(body, :nav, "id" => "p-lang")
    langs = get_tags(nav, :ul, "class" => "vector-menu-content-list")
    # 提取英文
    for lang in langs.children
        if lang.attributes["class"] == "interlanguage-link interwiki-en mw-list-item"
            en = lang.children[1].attributes["title"]
            return replace(en, " – 英语" => "", ' ' => '_')
        end
    end
    ""
end

function get_tags(body, symbol::Symbol, pair::Pair)
    for ele in PreOrderDFS(body)
        try
            tag(ele) == symbol && ele.attributes[pair[1]] == pair[2] && return ele
        catch
        end
    end
end

get_tags (generic function with 2 methods)